## Does Money Talk?
### An exploration of budget allocations and how they are (or are not) reflected in day-to-day services in Chicago
Tori Beck
CAPP 30239: Data Visualization for Policy Analysis

This notebook produces all of the final charts used in the static visualization project.  All data is housed in the `data` folder, including two files used to clean raw data (`clean_budgets.py` and `clean_library.py`).  These files have already been run and do not need to be used again in this notebook.  Most charts require a small amount of pre-processing before running, this code is included within the function or in the code chunk before.  Therefore, this file should be run sequentially. 

In [249]:
# Import all packages needed for pre-processing and visualizing data
import altair as alt
import pandas as pd
import numpy as np

In [267]:
# Define a theme and color scheme
alt.themes.enable("vox")
colors = ["#EAEAEA","#3BD4CF","#EA2B8B", "#F8C22C","#108D50","#00058F","#C994F0", "#FF6F6F","#921A80","#BD591B"]

In [268]:
# Read in budget data for 2019-2023
budgets = pd.read_csv("../data/budgets.csv")
budgets["year"] = budgets["year"].astype(str)
# Scale dollar amounts to billions of dollars to create cleaner look
budgets["ordinance"] = budgets["ordinance"]/1000000000

In [252]:
# Chart 1: City of Chicago Budget 2019-2023, all departments
def budget_by_year_area(df):
    # Define an "Other" category to separate out departments not being focused on
    df["department2"] = np.where(df["dept_num"].isin([57, 81, 84, 91, 41, 58, 59]), 
                                 df["department"], "All Other Departments")
    # Get sum of budgets for each department by year
    df = df.groupby(["year", "department2"], as_index = False)["ordinance"].sum()
    chart = alt.Chart(df, title = "City of Chicago Budget 2019-2023"
            ).mark_area().encode(
        x = alt.X("year").title("Year").axis(labelAngle = 0),
        y = alt.Y("ordinance").title("Budget (billions of $)"),
        color = alt.Color("department2").title("Department"),
        order = alt.Order("department2", sort = "descending")
    ).properties(height=500, 
                 width=500
    ).configure_legend(labelLimit= 0,
                        padding = 10,
                        cornerRadius = 5,
                        labelFontSize = 12,
                        symbolSize = 200,
                        titleFontSize = 14,
                        fillColor = "#FFF5DB"
    ).configure_range(
        category=alt.RangeScheme(colors)
)

    return chart

budget_by_year_area(budgets)

alt.Chart(...)

In [253]:
# Chart 2: Budget by year, only focusing on selected departments
def budget_by_year_dept(df):
    df = df[df["dept_num"].isin([57, 81, 84, 91, 41, 58, 59])]
    df = df.groupby(["year", "department"], as_index = False)["ordinance"].sum()
    
    chart = alt.Chart(df, title = "Selected Department Budgets 2019-2023"
            ).mark_line(point=True, size = 3).encode(
        x = alt.X("year").title("Year").axis(labelAngle = 0),
        y = alt.Y("ordinance").title("Budget (billons of $)"),
        color = alt.Color("department").title("Department")
    ).properties(width = 500, height = 400
    ).configure_legend(labelLimit= 0,
                       padding = 10,
                       cornerRadius = 5,
                       labelFontSize = 12,
                       symbolSize = 200,
                       titleFontSize = 14,
                       fillColor = "#FFF5DB"
    ).configure_range(
        category=alt.RangeScheme(colors[1:])
)

    return chart

budget_by_year_dept(budgets)

alt.Chart(...)

In [254]:
# Read in 311 request data, extract the year, and normalize column names
requests_311 = pd.read_csv("../data/311_requests.csv")
requests_311["by_year_created_date"] = requests_311["by_year_created_date"].str.extract(r"(\d{4})")
requests_311["OWNER_DEPARTMENT"] = requests_311["OWNER_DEPARTMENT"].str.replace(r"(.*(?=-)- )", "", regex = True)
requests_311 = requests_311.rename(columns = {"SR_NUMBER": "num_requests",
                                               "by_year_created_date": "year",
                                               "OWNER_DEPARTMENT": "department",
                                               "SR_TYPE": "type"})

In [255]:
# Define a dictionary to map differing department names to each other from the 311 dataset to the budget dataset
dept_dict = {"Department of Transportation": "Chicago Department of Transportation",
    "Animal Care and Control": "Chicago Animal Care and Control", 
    "Streets and Sanitation": "Department of Streets and Sanitation", 
    "Health": "Chicago Department of Public Health", 
    "Department of Water Management": "Department of Water Management", 
    "311 City Services": "Office of Emergency Management and Communications", 
    "Buildings": "Department of Buildings", "Aviation": "Chicago Department of Aviation",
    "Other": "Other"}

# Chart 3: Stacked bar chart showing which departments get the most 311 requests year over year
def stacked_bar_311(df):
    df["department2"] = np.where(df["department"].isin(["Department of Transportation",
       'Animal Care and Control', 'Streets and Sanitation', 
       'Health', 'Department of Water Management', '311 City Services', 
       'Buildings','Aviation']), df["department"], "Other")
    df["department2"] = df["department2"].map(dept_dict)
    df = df.groupby(["year", "department2"], as_index = False)["num_requests"].sum()

    chart = alt.Chart(df, title = "311 Service Requests by Department").mark_bar().encode(
        x=alt.X("year").title("Year").axis(labelAngle = 0),
        y = alt.Y("num_requests").stack("normalize").title("Percent of Requests"),
        color = alt.Color("department2").title("Department")
    ).properties(width = 400, height = 600
    ).configure_legend(labelLimit= 0,
                        padding = 10,
                        cornerRadius = 5,
                        labelFontSize = 12,
                        symbolSize = 200,
                        titleFontSize = 14,
                        fillColor = "#FFF5DB"
    ).configure_range(
        category=alt.RangeScheme(colors[1:])
)
    return chart

stacked_bar_311(requests_311)

alt.Chart(...)

In [256]:
# Chart 4: Scatterplots showing departmental budgets and number of 311 requests, faceted by year
def scatter_311_facet(df):
    df = df[df["department"].isin(["Department of Transportation",
       "Animal Care and Control", "Streets and Sanitation", 
       "Health", "Department of Water Management", "311 City Services", 
       "Buildings", "Aviation"])]
    df["department"] = df["department"].map(dept_dict)

    df = df.groupby(["year", "department"], as_index = False)["num_requests"].sum()
    # Subset the budget dataset to only the departments with the largest number of 311 requests
    budgets_311_subset =  budgets[budgets["department"].isin([
        "Chicago Department of Transportation",
       'Chicago Animal Care and Control', 'Department of Streets and Sanitation', 
       'Chicago Department of Public Health', 'Department of Water Management', 
       "Office of Emergency Management and Communications", 
       'Department of Buildings','Chicago Department of Aviation'])]
    budgets_311_subset = budgets_311_subset.groupby(["year", "department"], as_index = False)["ordinance"].sum()
    # Combine the 311 request data and budget data
    df = pd.merge(df, budgets_311_subset, how = "left", on = ["year", "department"])

    chart = alt.Chart(df, title = "311 Requests by Department Budget").mark_circle(fillOpacity = 0.8, size = 100).encode(
        x = alt.X("num_requests").title("# of 311 Requests"),
        y = alt.Y("ordinance").title("Budget (billions of $)"),
        color = alt.Color("department").title("Department"),
        facet = alt.Facet("year").title("Year")
    ).properties(width = 200, height = 200
    ).configure_legend(labelLimit= 0,
                        padding = 10,
                        cornerRadius = 5,
                        labelFontSize = 12,
                        symbolSize = 200,
                        titleFontSize = 14,
                        fillColor = "#FFF5DB"
    ).configure_range(
        category=alt.RangeScheme(colors[1:]))
    return chart

scatter_311_facet(requests_311)

/var/folders/n1/6wbb6bb96cs6326m2jt0gqkc0000gn/T/ipykernel_2332/31708591.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["department"] = df["department"].map(dept_dict)


alt.Chart(...)

In [257]:
# Read in the crime data set and subset the budget data to only CPD data
crimes = pd.read_csv("../data/crimes.csv")
crimes = crimes.rename(columns = {"ID": "num_crimes",
                                  "Primary Type": "prim_type",
                                  "Location Description": "location",
                                  "Arrest": "arrest",
                                  "Domestic": "domestic",
                                  "Community Area": "community_area",
                                  "Year": "year"})
crimes["year"] = crimes["year"].astype(str)
budgets_cpd =  budgets[budgets["department"] == "Chicago Police Department"]
budgets_cpd = budgets_cpd.groupby(["year"], as_index = False)["ordinance"].sum()

In [289]:
# Chart 5: Bubbleplot showing the # of crimes reported vs arrests made, along with budget year over year
def crime_cpd_budget_bubble(df):
    # Create new subset of data with only crimes ending in arrest
    arrests = df[df["arrest"] == True]
    arrests = arrests.groupby(["year"], as_index = False)["num_crimes"].sum()
    arrests = arrests.rename(columns = {"num_crimes": "num_arrests"})
    df = df.groupby(["year"], as_index = False)["num_crimes"].sum()

    df = df.join(arrests.set_index("year"), on = "year")
    df = df.join(budgets_cpd.set_index("year"), on = "year")

    chart = alt.Chart(df, title = "Chicago Crime Statistics and CPD Budget 2019-2023").mark_circle(fillOpacity = 0.8).encode(
        x = alt.X("num_arrests").title("# Crimes Reported Ending in Arrest"),
        y = alt.Y("num_crimes").title("# Crimes Reported").scale(zero=False, padding=1),
        color = alt.Color("year").title("Year"),
        size = alt.Size("ordinance").title("Budget (billions of $)").scale(domain = [1.6, 1.8])
    ).configure_legend(labelLimit= 0,
                        padding = 10,
                        cornerRadius = 5,
                        labelFontSize = 12,
                        symbolSize = 200,
                        titleFontSize = 14,
                        fillColor = "#FFF5DB",
                        symbolType = "circle"
    ).configure_range(
        category=alt.RangeScheme(colors[1:])
)
    return chart

crime_cpd_budget_bubble(crimes)

alt.Chart(...)

In [259]:
# Read in police sentiment data and extract year
police_sentiment = pd.read_csv("../data/police_sentiment_scores.csv")
police_sentiment["by_year_start_date"] = police_sentiment["by_year_start_date"].str.extract(r"(\d{4})")
police_sentiment = police_sentiment.rename(columns = {"by_year_start_date": "year",
                                                      "TRUST": "Trust",
                                                      "SAFETY": "Safety"})

In [260]:
# Chart 6: Pairwise scatterplots showing both measures of police sentiment alongside budget
def scatter_matrix_cpd(df):
    crimes_df = crimes.groupby(["year"], as_index = False)["num_crimes"].sum()
    safety = df.groupby(["year"], as_index = False)["Safety"].mean()
    df = df.groupby(["year"], as_index = False)["Trust"].mean()
    df = df.join(safety.set_index("year"), on = "year")
    df = df.join(budgets_cpd.set_index("year"), on = "year")
    df = df.join(crimes_df.set_index("year"), on = "year")

    chart = alt.Chart(df, title = "CPD Sentiment").mark_circle(size = 100, fillOpacity = 0.8).encode(
        x = alt.X(alt.repeat("column"), type='quantitative').scale(zero=False, padding=1).title("Budget (billions of $)"),
        y = alt.Y(alt.repeat("row"), type='quantitative').scale(zero=False, padding=1),
        color = alt.Color("year").title("Year")
    ).properties(width = 200, height = 200
    ).repeat(
        row=["Safety", "Trust"],
        column=["ordinance"]
    ).configure_legend(labelLimit= 0,
                        padding = 10,
                        cornerRadius = 5,
                        labelFontSize = 12,
                        titleFontSize = 14,
                        fillColor = "#FFF5DB"
    ).resolve_axis(
    ).configure_range(
        category=alt.RangeScheme(colors[1:])
)

    return chart

scatter_matrix_cpd(police_sentiment)

alt.RepeatChart(...)

In [261]:
# Read in library visitors data
library_visitors = pd.read_csv("../data/library_visitors.csv")
library_visitors["year"] = library_visitors["year"].astype(str)

In [290]:
# Chart 7: Heatmap showing number of visitors to CPL locations per month from 2019-2023
def heatmap_library_visitors(df):
    df = df[df["month"] != "YTD"]
    df = df.groupby(["year", "month"], as_index = False)["num_visitors"].sum()

    chart = alt.Chart(df, title = "Chicago Public Library Visits 2019-2023").mark_rect().encode(
        x = alt.X("year").title("Year").axis(labelAngle = 0),
        y = alt.Y("month", sort = ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 
                                   'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 
                                   'NOVEMBER', 'DECEMBER']).title("Month"),
        color = alt.Color("num_visitors").title("# Visitors").scale(scheme = "purplebluegreen")
    ).properties(width = 500, height = 400
    ).configure_legend(labelLimit= 0,
                        padding = 10,
                        cornerRadius = 5,
                        labelFontSize = 12,
                        titleFontSize = 14,
                        fillColor = "#FFF5DB"
    )

    return chart

heatmap_library_visitors(library_visitors)

alt.Chart(...)

In [263]:
# Subset budget data to only CPL budget
budgets_cpl =  budgets[budgets["department"] == "Chicago Public Library"]
budgets_cpl = budgets_cpl.groupby(["year"], as_index = False)["ordinance"].sum()


In [264]:
# Chart 8: Bar graph showing change in CPL visitors alongside budget changes
def concat_chart_library_budget(df):
    df = df.groupby(["year"], as_index = False)["num_visitors"].sum()
    df = df.join(budgets_cpl.set_index("year"), on = "year")

    upper = alt.Chart(df, title = "Chicago Public Library Visitors 2019-2023").mark_bar(color = "#00058F").encode(
        x = alt.X("year").title("Year").axis(labelAngle = 0),
        y = alt.Y("num_visitors").title("# Visitors"),
    ).properties(width = 600, height = 400
    )

    lower = alt.Chart(df, title = "Chicago Public Library Budget").mark_line(
        color = "#7570B3").encode(
        x = alt.X("year").title("Year").axis(labelAngle = 0),
        y = alt.Y("ordinance").title("Budget (billions of $)"),
    ).properties(width = 600, height = 100
    )

    chart = alt.vconcat(upper, lower)

    return chart

concat_chart_library_budget(library_visitors)

alt.VConcatChart(...)

In [265]:
# Read in circulation data
library_circulation = pd.read_csv("../data/library_circulation.csv")
library_circulation["year"] = library_circulation["year"].astype(str)

# Chart 9: Heatmap showing CPL circulation numbers per month from 2019-2023
def heatmap_library_circulation(df):
    df = df[df["month"] != "YTD"]
    df = df.groupby(["year", "month"], as_index = False)["circulation_num"].sum()

    chart = alt.Chart(df, title = "Chicago Public Library Circulation 2019-2023").mark_rect().encode(
        x = alt.X("year").title("Year").axis(labelAngle = 0),
        y = alt.Y("month", sort = ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 
                                   'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 
                                   'NOVEMBER', 'DECEMBER']).title("Month"),
        color = alt.Color("circulation_num").title("Circulation").scale(scheme = "purplebluegreen")
    ).properties(width = 500, height = 400
    ).configure_legend(labelLimit= 0,
                        padding = 10,
                        cornerRadius = 5,
                        labelFontSize = 12,
                        titleFontSize = 14,
                        fillColor = "#FFF5DB"
    )

    return chart

heatmap_library_circulation(library_circulation)

alt.Chart(...)

In [266]:
# Chart 10: Pairwise scatterplots showing associations between CPL visitors, circulation, and budget year over year
def cpl_pairwise_plots(df):
    df = df.groupby(["year"], as_index = False)["circulation_num"].sum()
    visitors = library_visitors.groupby(["year"], as_index = False)["num_visitors"].sum()
    df = df.join(visitors.set_index("year"), on = "year")
    df = df.join(budgets_cpl.set_index("year"), on = "year")

    scatter1 = alt.Chart(df, title = "Chicago Public Library Visitors vs Circulation 2019-2023").mark_circle(fillOpacity = 0.8, size = 100).encode(
        x = alt.X("num_visitors").title("# of Visitors").scale(zero=False, padding=1),
        y = alt.Y("circulation_num").title("Circulation").scale(zero=False, padding=1),
        color = alt.Color("year").title("Year")
    )

    scatter2 = alt.Chart(df, title = "Chicago Public Library Visitors vs Budget 2019-2023").mark_circle(fillOpacity = 0.8, size = 100).encode(
        x = alt.X("num_visitors").title("# of Visitors").scale(zero=False, padding=1),
        y = alt.Y("ordinance").title("Budget (billions of $)").scale(zero=False, padding=1),
        color = alt.Color("year").title("Year")
    )

    scatter3 = alt.Chart(df, title = "Chicago Public Circulation vs Budget 2019-2023").mark_circle(fillOpacity = 0.8, size = 100).encode(
        x = alt.X("circulation_num").title("Circulation").scale(zero=False, padding=1),
        y = alt.Y("ordinance").title("Budget (billions of $)").scale(zero=False, padding=1),
        color = alt.Color("year").title("Year")
    )
    
    chart = alt.hconcat(scatter1, scatter2, scatter3)

    return chart.configure_legend(labelLimit= 0,
                        padding = 10,
                        cornerRadius = 5,
                        labelFontSize = 12,
                        symbolSize = 200,
                        titleFontSize = 14,
                        fillColor = "#FFF5DB"
    ).resolve_axis(
    ).configure_range(
        category=alt.RangeScheme(colors[1:])
)

cpl_pairwise_plots(library_circulation)

alt.HConcatChart(...)